In [233]:
import pandas as pd
import ast

In [234]:
# define the path for the input csv files
RAW_DATA_PATH: str = 'data/preprocessed_datasets/non_cpd/dataset_warum_binary_unbalanced.csv'
# define the path for the output parquet files
EXPORTED_DATA_PATH: str = 'data/parquet_files'

TRAIN_SPLIT: float = 0.8
RANDOM_STATE: int = 42

In [235]:
raw: pd.DataFrame = (
    pd.read_csv(
        RAW_DATA_PATH,
        skiprows=[1],
        index_col=[0],
        dtype={
            'text': 'string',
            'label': 'category'
        },
        converters={
            'span': ast.literal_eval
        }
    )
    .rename_axis('idx')
)
raw

,text,label,span
idx,,,
1f09f195-f651-433a-8e43-8c7a47d07558,"„Warum sollen wir Kredite nehmen, deren Tilgun...",corona,"[1, 6]"
cfbc74ca-cc3e-47e6-9d7d-bdd23197c8d5,Warum wir keinen Urlaub nehmen?,corona,"[0, 5]"
a0c1f82a-9b4a-4b98-beb1-ab3bfeef0f13,"Warum wir nicht Oma und Opa zur Pflicht rufen,...",corona,"[0, 5]"
3aa2021c-36dd-483f-8e8f-0cf2b56ac475,Warum der Solardeckel nicht schon längst wie a...,corona,"[0, 5]"
48c4a9ce-0ff8-4266-a051-ce0bc723fdad,Wie Sie ihr Geld erfolgreich anlegen und warum...,corona,"[41, 46]"
...,...,...,...
12645839-5b80-4319-a30f-8057f48aaa79,Garantiert ein Aspekt der erklärt warum gerade...,web2,"[34, 39]"
ced570a6-e3a9-4579-9370-25426879c7ac,Warum nicht als zentrale Anlaufstelle für Apps...,web2,"[0, 5]"
01727803-1d5c-4ffd-9a6f-7cea70bd2a34,Warum der Mann auf die Gleise des Bahnsteigs 5...,web2,"[0, 5]"


In [236]:
raw.dtypes

text     string[python]
label          category
span             object
dtype: object

In [237]:
raw.label.unique()

['corona', 'web2']
Categories (2, object): ['corona', 'web2']

In [238]:
raw.label.value_counts()

label
corona    4996
web2      4962
Name: count, dtype: int64

In [239]:
(
    raw
    .apply(lambda row: row.text[row.span[0]:row.span[1]], axis=1)
    .value_counts()
)

warum    5539
Warum    4414
            4
AME?        1
Name: count, dtype: int64

In [240]:
train_split = raw.sample(
    frac=TRAIN_SPLIT, 
    random_state=RANDOM_STATE
)
# define the name for the resulting train parquet file
train_split.to_parquet(f'{EXPORTED_DATA_PATH}/warum_bu_train.parquet')

In [241]:
test_split = raw.loc[
    raw.index.difference(train_split.index)
]
# define the name for the resulting test parquet file
test_split.to_parquet(f'{EXPORTED_DATA_PATH}/warum_bu_test.parquet')